In [1]:
import numpy as np
import h5py
import tables
import matplotlib.pyplot as plt
from collections import OrderedDict
import tables
import scipy.stats
import pandas as pd
from matplotlib.font_manager import FontProperties

In [2]:
linestyles = OrderedDict(
    [('solid',               (0, ())),
     ('loosely dotted',      (0, (1, 10))),
     ('dotted',              (0, (1, 5))),
     ('densely dotted',      (0, (1, 1))),

     ('loosely dashed',      (0, (5, 10))),
     ('dashed',              (0, (5, 5))),
     ('densely dashed',      (0, (5, 1))),

     ('loosely dashdotted',  (0, (3, 10, 1, 10))),
     ('dashdotted',          (0, (3, 5, 1, 5))),
     ('densely dashdotted',  (0, (3, 1, 1, 1))),

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))])

In [3]:
# Initializing Source Voxels
sourceX, sourceY, sourceZ = np.mgrid[ -299:301:2, -1:1:2, -1:1:2 ]
xpos = sourceX[:, 0, 0]

In [4]:
solid = ( 74 * 74 ) / ( 4 * np.pi * 40 ** 2 + ( 2 * 74 * 74 ) )

In [15]:
# Loading System Response
a = 180
b = 90
m = 15

div = '/Users/eframe/Downloads/div_b' + str(b) + '_m' + str(m) + '_v20.h5'
con = '/Users/eframe/Downloads/con_b' + str(b) + '_m' + str(m) + '_v20.h5'
par = '/Users/eframe/Downloads/par_b' + str(b) + '_m' + str(m) + '_v20.h5'

mat_div = tables.open_file( div, 'r').root.matrix.read()
mat_con = tables.open_file( con, 'r').root.matrix.read()
mat_par = tables.open_file( par, 'r').root.matrix.read()

sens_div = mat_div.sum( axis = 1 ) * 1000 
sens_con = mat_con.sum( axis = 1 ) * 1000 
sens_par = mat_par.sum( axis = 1 ) * 1000 

mask_div = sens_div > 0
mask_con = sens_con > 0
mask_par = sens_par > 0

sens_div_nozero = sens_div[mask_div]
sens_con_nozero = sens_con[mask_con]
sens_par_nozero = sens_par[mask_par]

In [16]:
# fully-coded field of view
m = 1 + b / a

ld = 74
lm = 128 

fcfv = ( m * lm - ld ) / ( m - 1 )
lim = fcfv / 2

x =  ( ld + ( lm - ld ) / 2 ) * ( b + a ) / b - ld
val = 2 * x + ld
pcfv = ( val - fcfv ) / 2 + 2

mask = ( xpos > -lim-pcfv-1 ) & ( xpos < lim + pcfv )

In [17]:
%matplotlib qt
font = FontProperties()
font.set_name('Times New Roman')
fig, ax = plt.subplots()
ax.plot( xpos[mask], sens_div[mask], linestyle = linestyles['solid'], linewidth=7, label='Diverging')
ax.plot( xpos[mask], sens_con[mask], linestyle = linestyles['densely dashed'], linewidth=7, label='Converging')
ax.plot( xpos[mask], sens_par[mask], linestyle = linestyles['densely dotted'], linewidth=7, label='Parallel')
ax.minorticks_on()
ax.set_xlabel('Distance from FoV Center [mm]', fontsize=50, fontproperties=font)
ax.set_ylabel(r'Sensitivity $\quad \left[\frac{cps}{kBq\cdot122~keV}\right]$', fontsize=50, fontproperties=font)
ax.axvspan(-lim, lim,  alpha=0.25, label="FCFV")
ax.axvspan(-lim-pcfv-1, -lim, color='r', alpha=0.15, label="PCFV")
ax.axvspan(lim, lim+pcfv, color='r', alpha=0.15)
ax.tick_params(which='major', length=10, width=2, direction='in', labelsize=50)
ax.tick_params(which='minor', length=5, width=2, direction='in', labelsize=50)
# legend =ax.legend(prop={"family":"Times New Roman", 'size': 40}, 
#                   ncol=2, loc='center', bbox_to_anchor=(0.8, 0.95), facecolor='white', framealpha=1)
legend =ax.legend(prop={"family":"Times New Roman", 'size': 40}, edgecolor='white',facecolor='white', framealpha=1)
ax.yaxis.get_offset_text().set_fontsize(40)
ax.yaxis.get_offset_text().set_family("Times New Roman")
ax.set_ylim(0.00001)
# ax.grid(which='both')
ax.set_xlim(-300, 300)
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)    
plt.show()

In [ ]:
# Calculating contrast
cont_div = mat_div.reshape( (sourceX.shape[0], sourceX.shape[1], 5476) )[:,0,:]
cont_con = mat_con.reshape( (sourceX.shape[0], sourceX.shape[1], 5476) )[:,0,:]
cont_par = mat_par.reshape( (sourceX.shape[0], sourceX.shape[1], 5476) )[:,0,:]

In [ ]:
%matplotlib inline
grid = cont_con[150,:].reshape((74,74)) 
fig, ax = plt.subplots()
im = ax.pcolormesh(grid.T)
ax.set_xlabel('x (mm)', fontsize=20)
ax.set_ylabel('y (mm)', fontsize=20)
ax.tick_params(labelsize=20)
cbar = plt.colorbar(im)
cbar.set_label(label='counts / pixel', rotation=270, fontsize=20, labelpad=20)
cbar.ax.tick_params(labelsize=20)
plt.show()

In [ ]:
%matplotlib inline
grid = cont_div[150,:].reshape((74,74)) 
fig, ax = plt.subplots()
im = ax.pcolormesh(grid.T)
ax.set_xlabel('x (mm)', fontsize=20)
ax.set_ylabel('y (mm)', fontsize=20)
ax.tick_params(labelsize=20)
cbar = plt.colorbar(im)
cbar.set_label(label='counts / pixel', rotation=270, fontsize=20, labelpad=20)
cbar.ax.tick_params(labelsize=20)
plt.show()

In [ ]:
array = np.array( [ cont_div, 
                   cont_con, 
                   cont_par ] )
contrast = []
for i in np.arange( array.shape[0] ):
    cont = []
    projections = array[i, :]
    for j in np.arange( projections.shape[0] ):
        proj = projections[j] 
        N = len( proj )
        mu = np.mean( proj )
        std = np.std( proj )
        kurt = scipy.stats.kurtosis( proj )
        cont.append( std / max(projections[j]) )
    contrast.append( cont )

In [ ]:
%matplotlib inline
font = FontProperties()
font.set_name('Times New Roman')
fig, ax = plt.subplots()
ax.plot( xpos[mask], np.array(contrast[0])[mask], linestyle = linestyles['solid'], linewidth=7, label='Diverging')
ax.plot( xpos[mask], np.array(contrast[1])[mask], linestyle = linestyles['densely dashed'], linewidth=7, label='Converging')
ax.plot( xpos[mask], np.array(contrast[2])[mask], linestyle = linestyles['densely dotted'], linewidth=7, label='Parallel')
ax.minorticks_on()
ax.axvspan(-lim, lim,  alpha=0.25, label="FCFV")
ax.axvspan(-lim-pcfv-1, -lim, color='r', alpha=0.15, label="PCFV")
ax.axvspan(lim, lim+pcfv, color='r', alpha=0.15)
ax.set_xlabel('Distance from FoV Center [mm]', fontsize=40, fontproperties=font, labelpad=10)
ax.set_ylabel(r'Coding Contrast', fontsize=40, fontproperties=font, labelpad=15)
ax.tick_params(which='major', length=10, width=2, direction='in', labelsize=40)
ax.tick_params(which='minor', length=5, width=2, direction='in', labelsize=40)
# legend =ax.legend(prop={"family":"Times New Roman", 'size': 30}, 
#                   ncol=2, loc='center', bbox_to_anchor=(0.85, 1.05), facecolor='white', framealpha=1)
ax.yaxis.get_offset_text().set_fontsize(30)
ax.yaxis.get_offset_text().set_family("Times New Roman")
# ax.set_ylim(0.001, 0.5)
ax.set_xlim(-300, 300)
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(40)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(40)  
ax.grid(which='both')
plt.show()